In [37]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import matplotlib.tri as mtri


### 0. Cleanup Mesh

In [2]:
def remove_redundant_vertices(mesh):
    # Get the vertices and triangles
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    
    # Identify unique vertex indices used in the triangles
    unique_vertex_indices = np.unique(triangles)
    
    # Create a mapping from old vertex indices to new ones
    old_to_new_indices = {old_idx: new_idx for new_idx, old_idx in enumerate(unique_vertex_indices)}
    
    # Create a new list of vertices that are only the used ones
    new_vertices = vertices[unique_vertex_indices]
    
    # Update triangle indices to the new vertex indices
    new_triangles = np.array([[old_to_new_indices[old_idx] for old_idx in triangle] for triangle in triangles])
    
    # Create a new mesh with the cleaned vertices and updated triangles
    cleaned_mesh = o3d.geometry.TriangleMesh()
    cleaned_mesh.vertices = o3d.utility.Vector3dVector(new_vertices)
    cleaned_mesh.triangles = o3d.utility.Vector3iVector(new_triangles)
    
    # Copy colors, normals, etc. if they exist
    if mesh.has_vertex_colors():
        cleaned_mesh.vertex_colors = o3d.utility.Vector3dVector(np.asarray(mesh.vertex_colors)[unique_vertex_indices])
    if mesh.has_vertex_normals():
        cleaned_mesh.vertex_normals = o3d.utility.Vector3dVector(np.asarray(mesh.vertex_normals)[unique_vertex_indices])
    
    return cleaned_mesh

In [9]:
# Load the mesh
mesh = o3d.io.read_triangle_mesh(r'D:\sunny\Codes\DPS\data_teethseg\origin\000101_origin.ply')
# print(np.array(mesh.vertices).shape)
# print(np.array(mesh.triangles).shape)

# Remove redundant vertices
intraoral_mesh = remove_redundant_vertices(mesh)
# print(np.array(mesh.vertices).shape)
# print(np.array(mesh.triangles).shape)

# Visualize the cleaned mesh
o3d.visualization.draw_geometries([intraoral_mesh])

### 1. Create Circle and Extrude to Cylinders as Projection Screens

In [10]:
# Function to create a circle in the x-z plane
def create_circle(radius, n_points=100):
    theta = np.linspace(0, 2 * np.pi, n_points)
    x = radius * np.cos(theta)
    z = radius * np.sin(theta)
    points = np.vstack((x, np.zeros(n_points), z)).T
    lines = np.column_stack((np.arange(n_points), np.roll(np.arange(n_points), -1)))
    return points, lines

# Function to extrude the circles in the y direction to create cylinders
def extrude_circle(points, extrusion_length_pos, extrusion_length_neg):
    extruded_points = []
    for y in [extrusion_length_neg, extrusion_length_pos]:
        extruded_points.append(points + np.array([0, y, 0]))
    extruded_points = np.vstack(extruded_points)
    
    # Create triangular faces for the extrusion
    n_points = len(points)
    faces = []
    for i in range(n_points):
        next_i = (i + 1) % n_points  # Ensure circular connection
        # Create two triangles for each face of the cylinder
        faces.append([i, next_i, next_i + n_points])
        faces.append([i, next_i + n_points, i + n_points])
    
    return extruded_points, faces

Calculate cylinder parameters based on mesh geometry:

In [12]:

# Calculate the radii
distances = np.linalg.norm(np.asarray(intraoral_mesh.vertices)[:, [0, 2]], axis=1)
outer_radius = np.max(distances)
inner_radius = np.min(distances)
n_points = 100

# Calculate the extrusion length
y_min = np.min(np.asarray(intraoral_mesh.vertices)[:, 1])
y_max = np.max(np.asarray(intraoral_mesh.vertices)[:, 1])
extrusion_length_pos = y_max
extrusion_length_neg = y_min
print(f'Extrusion length: {extrusion_length_pos} (pos), {extrusion_length_neg} (neg)')


Extrusion length: 10.682730266372726 (pos), -0.62988083750371 (neg)


In [13]:
# Create the circles
circle_outer_points, circle_outer_lines = create_circle(outer_radius, n_points)
circle_inner_points, circle_inner_lines = create_circle(inner_radius, n_points)

# Extrude the circles to create cylinders
cylinder_outer_points, cylinder_outer_faces = extrude_circle(circle_outer_points, extrusion_length_pos, extrusion_length_neg)
cylinder_inner_points, cylinder_inner_faces = extrude_circle(circle_inner_points, extrusion_length_pos, extrusion_length_neg)


Create meshes for visualization:

In [14]:
# Create meshes for the extruded cylinders
cylinder_outer = o3d.geometry.TriangleMesh()
cylinder_outer.vertices = o3d.utility.Vector3dVector(cylinder_outer_points)
cylinder_outer.triangles = o3d.utility.Vector3iVector(cylinder_outer_faces)
cylinder_outer.paint_uniform_color([0.5, 0.5, 1.0])

cylinder_inner = o3d.geometry.TriangleMesh()
cylinder_inner.vertices = o3d.utility.Vector3dVector(cylinder_inner_points)
cylinder_inner.triangles = o3d.utility.Vector3iVector(cylinder_inner_faces)
cylinder_inner.paint_uniform_color([0.5, 1.0, 0.5])

# Create coordinate axes
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=[0, 0, 0])

# Create grid lines in the xz plane
grid_x = np.linspace(-100, 100, 201)
grid_z = np.linspace(-100, 100, 201)
grid_points = np.array([[x, 0, z] for x in grid_x for z in grid_z])
grid_lines = []
for i, x in enumerate(grid_x):
    grid_lines.append([i * len(grid_z), (i + 1) * len(grid_z) - 1])
for i, z in enumerate(grid_z):
    grid_lines.append([i, len(grid_x) * len(grid_z) - len(grid_x) + i])
grid = o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(grid_points),
                            lines=o3d.utility.Vector2iVector(grid_lines))
grid.paint_uniform_color([0.8, 0.8, 0.8])

# Visualize the splines, curved planes, coordinate axes, and grid
o3d.visualization.draw_geometries([intraoral_mesh, cylinder_outer, cylinder_inner, coordinate_frame, grid])

### 2. Projections

In [16]:
# Calculate vertex normals
intraoral_mesh.compute_vertex_normals()
vertex_normals = np.asarray(intraoral_mesh.vertex_normals)
vertices = np.asarray(intraoral_mesh.vertices)

# Classify vertices as inward or outward-facing
outward_facing_mask = np.array([np.dot(normal, vertex) > 0 for normal, vertex in zip(vertex_normals, vertices)])
inward_facing_mask = np.array([np.dot(normal, vertex) < 0 for normal, vertex in zip(vertex_normals, vertices)])

outward_facing_vertices = vertices[outward_facing_mask]
inward_facing_vertices = vertices[inward_facing_mask]

In [18]:
# Project vertices onto the respective cylinders
def project_onto_cylinder(vertices, radius):
    projected_vertices = vertices.copy()
    for i in range(vertices.shape[0]):
        x, y, z = vertices[i]
        theta = np.arctan2(z, x)
        projected_vertices[i] = [radius * np.cos(theta), y, radius * np.sin(theta)]
    return projected_vertices

In [24]:
projected_outward_vertices = project_onto_cylinder(outward_facing_vertices, outer_radius)
projected_inward_vertices = project_onto_cylinder(inward_facing_vertices, inner_radius)

# Create meshes for the projected vertices
projected_outward_mesh = o3d.geometry.PointCloud()
projected_outward_mesh.points = o3d.utility.Vector3dVector(projected_outward_vertices)
projected_outward_mesh.paint_uniform_color([1, 0, 0])

projected_inward_mesh = o3d.geometry.PointCloud()
projected_inward_mesh.points = o3d.utility.Vector3dVector(projected_inward_vertices)
projected_inward_mesh.paint_uniform_color([1, 0, 0])

PointCloud with 9641 points.

In [25]:
# Preserve indices and connectivity
# Create a new mesh with projected vertices, keeping the same faces
new_mesh = copy.deepcopy(intraoral_mesh)
new_vertices = np.asarray(new_mesh.vertices)
new_vertices[outward_facing_mask] = projected_outward_vertices
new_vertices[inward_facing_mask] = projected_inward_vertices
new_mesh.vertices = o3d.utility.Vector3dVector(new_vertices)

In [26]:
# Visualize the original mesh, projected meshes, and coordinate axes
o3d.visualization.draw_geometries([intraoral_mesh, projected_outward_mesh, projected_inward_mesh, coordinate_frame, cylinder_outer, cylinder_inner])

### 3. Visualize with 2D Flattening

In [43]:
# Flatten the projected vertices into 2D planes
def flatten_to_2d(vertices, radius):
    flattened_vertices = []
    for i in range(vertices.shape[0]):
        x, y, z = vertices[i]
        theta = np.arctan2(z, x)
        flattened_vertices.append([theta * radius, y])
    return np.array(flattened_vertices)

# # Preserve the connectivity and assign colors based on the minimal RGB values among vertices of each face
# def create_flat_mesh(flattened_vertices, faces, colors):
#     flat_vertices = flattened_vertices[:, :2]  # Only keep the x and y coordinates
#     face_colors = np.zeros((len(faces), 3))
#     for i, face in enumerate(faces):
#         face_colors[i] = np.min(colors[face], axis=0)
#     return flat_vertices, faces, face_colors




In [44]:
# Flatten the projected vertices into 2D planes
flattened_outward_vertices = flatten_to_2d(projected_outward_vertices, outer_radius)
flattened_inward_vertices = flatten_to_2d(projected_inward_vertices, inner_radius)

# Create the 2D meshes with the original connectivity
faces = np.asarray(intraoral_mesh.triangles)
colors = np.asarray(intraoral_mesh.vertex_colors)

# Create two new arrays for the flattened projections
flat_outer_vertices = []
flat_outer_faces = []
flat_outer_colors = []

flat_inner_vertices = []
flat_inner_faces = []
flat_inner_colors = []


# Create a mapping from the original indices to the new indices
outward_index_mapping = {old_idx: new_idx for new_idx, old_idx in enumerate(np.where(outward_facing_mask)[0])}
inward_index_mapping = {old_idx: new_idx for new_idx, old_idx in enumerate(np.where(inward_facing_mask)[0])}


# Preserve the connectivity and assign colors based on the minimal RGB values among vertices of each face
for face in faces:
    if all(outward_facing_mask[face]):
        new_face = [outward_index_mapping[idx] for idx in face]
        flat_outer_faces.append(new_face)
        face_colors = colors[face]
        flat_outer_colors.append(np.min(face_colors, axis=0))
    elif all(inward_facing_mask[face]):
        new_face = [inward_index_mapping[idx] for idx in face]
        flat_inner_faces.append(new_face)
        face_colors = colors[face]
        flat_inner_colors.append(np.min(face_colors, axis=0))

flat_outer_faces = np.array(flat_outer_faces)
flat_inner_faces = np.array(flat_inner_faces)
flat_outer_colors = np.array(flat_outer_colors).reshape(-1, 3)
flat_inner_colors = np.array(flat_inner_colors).reshape(-1, 3)


# Assuming `create_flat_mesh` and the rest of the code is correct
# Ensure face_colors have correct shape
flat_outer_colors = np.min(colors[outward_facing_mask][faces], axis=1)
flat_inner_colors = np.min(colors[inward_facing_mask][faces], axis=1)

IndexError: index 10974 is out of bounds for axis 0 with size 10974

In [45]:
# Plotting function with larger plot size and no edges
def plot_2d_mesh(vertices, faces, face_colors):
    fig, ax = plt.subplots(figsize=(10, 10))
    triangulation = mtri.Triangulation(vertices[:, 0], vertices[:, 1], faces)
    ax.tripcolor(triangulation, facecolors=face_colors, edgecolors='none')
    ax.set_aspect('equal')
    plt.show()
    
plot_2d_mesh(flattened_outward_vertices, flat_outer_faces, flat_outer_colors)
plot_2d_mesh(flattened_inward_vertices, flat_inner_faces, flat_inner_colors)

ValueError: Collections can only map rank 1 arrays

<Figure size 1000x1000 with 1 Axes>

ValueError: Collections can only map rank 1 arrays

<Figure size 1000x1000 with 1 Axes>